In [13]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNWithMeans
# Step 1: Prepare the restaurant dataset
ratings_dict = { 
    "item": [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 3],  # Restaurants
    "user": ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'D'], 
    "rating": [5, 4, 3, 2, 1, 4, 3, 5, 2, 3, 2, 3, 5, 4, 2, 4],  # Ratings for restaurants
}
# Convert the dictionary to a DataFrame
df = pd.DataFrame(ratings_dict)
# Use the Reader class to define the rating scale
reader = Reader(rating_scale=(1, 5))
# Load the data into the Surprise library format
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
# Step 2: Implement Item-Based Collaborative Filtering
# Define the similarity options for KNN
sim_options = {
    "name": "cosine",  # Use cosine similarity
    "user_based": False,  # Item-based filtering (user_based=False)
}
# Initialize the KNNWithMeans algorithm
algo = KNNWithMeans(sim_options=sim_options)
# Build the training set and fit the model
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
# Step 3: Predict Ratings and Recommend Restaurants
# Function to recommend top N restaurants for a user
def recommend_restaurants(user_id, n_recommendations=3):
    # Get a list of all items (restaurants)
    all_items = set(df['item'])
    # Get the restaurants the user has already rated
    rated_items = set(df[df['user'] == user_id]['item'])
    # Print out the rated items for the user (debugging)
    print(f"Rated restaurants for user '{user_id}': {rated_items}")
    # Find the unrated restaurants
    unrated_items = all_items - rated_items
    print(f"Unrated restaurants for user '{user_id}': {unrated_items}")
    # If there are no unrated restaurants, return a message
    if not unrated_items:
        print(f"User '{user_id}' has rated all available restaurants.")
        return []
    # Predict ratings for all restaurants that the user hasn't rated
    predictions = []
    for item in unrated_items:
        prediction = algo.predict(user_id, item)
        print(f"Predicted rating for restaurant {item}: {prediction.est}")  # Debugging
        predictions.append((item, prediction.est))
    # Sort the predictions by the estimated rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)
    # Return the top N recommendations
    return predictions[:n_recommendations]
# Example: Recommend top 3 restaurants for user 'B'
recommendations = recommend_restaurants('D', n_recommendations=3)
if recommendations:
    print("\nTop 3 recommended restaurants for user 'D':")
    for restaurant, rating in recommendations:
        print(f"Restaurant {restaurant} with predicted rating {rating:.2f}")
else:
    print("No recommendations available.")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Rated restaurants for user 'D': {3}
Unrated restaurants for user 'D': {1, 2, 4, 5}
Predicted rating for restaurant 1: 3.4166666666666665
Predicted rating for restaurant 2: 3.0833333333333335
Predicted rating for restaurant 4: 2.4166666666666665
Predicted rating for restaurant 5: 1.75

Top 3 recommended restaurants for user 'D':
Restaurant 1 with predicted rating 3.42
Restaurant 2 with predicted rating 3.08
Restaurant 4 with predicted rating 2.42
